# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")
import pickle

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BokaiXu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BokaiXu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BokaiXu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterRecord.db')
df = pd.read_sql_table('DisasterRecord',engine)
X = df['message'].values
y = df.drop(['id','message','genre'],axis=1).values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    This is the function used to convert text to a list of words.
    """
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Build a machine learning pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [6]:
# train the data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000217E68C7E50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# classify test data with the model
y_pred=pipeline.predict(X_test)

In [8]:
# output the classification report
columns=df.drop(['id','message','genre'],axis=1).columns
for i in range(36):
    print(columns[i])
    print('accuracy_score ',accuracy_score(y_test.T[i],y_pred.T[i]))
    print('classification_report \n', classification_report(y_test.T[i],y_pred.T[i]))

related
accuracy_score  0.8162746344564527
classification_report 
               precision    recall  f1-score   support

           0       0.72      0.42      0.53      1870
           1       0.84      0.95      0.89      5934
           2       0.38      0.41      0.40        61

    accuracy                           0.82      7865
   macro avg       0.65      0.59      0.61      7865
weighted avg       0.80      0.82      0.80      7865

request
accuracy_score  0.8989192625556262
classification_report 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      6545
           1       0.83      0.50      0.63      1320

    accuracy                           0.90      7865
   macro avg       0.87      0.74      0.78      7865
weighted avg       0.89      0.90      0.89      7865

offer
accuracy_score  0.9952956134774317
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00     

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
# Grid Search
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

In [10]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  29.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.9s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  29.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  28.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  28.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, total=  28.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, c

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, total=  45.4s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, total=  45.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, total=  47.6s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5, total=  47.3s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_spl

[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, total=  46.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, total=  46.4s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, total=  46.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, total=  46.9s
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_sp

[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, total=  37.8s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, total=  37.1s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, total=  36.9s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, total=  38.1s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_sp

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 68.2min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000217E68C7E50>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5, 10],
                         'clf__estimator__n_estimators': [10, 25],
                         'tfidf__use_idf': [True, False],
                         'vect__min_df': [1, 5]},
             verbose=2)

In [11]:
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(min_df=5,
                                 tokenizer=<function tokenize at 0x00000217E68C7E50>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=25)))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Predcit test data with the best model found by cv
y_pred_best=cv.predict(X_test)

In [13]:
# output the classification report
for i in range(36):
    print(columns[i])
    print('accuracy_score ',accuracy_score(y_test.T[i],y_pred_best.T[i]))
    print('classification_report \n', classification_report(y_test.T[i],y_pred_best.T[i]))

related
accuracy_score  0.8115702479338843
classification_report 
               precision    recall  f1-score   support

           0       0.68      0.44      0.54      1870
           1       0.84      0.93      0.88      5934
           2       0.36      0.33      0.34        61

    accuracy                           0.81      7865
   macro avg       0.63      0.57      0.59      7865
weighted avg       0.80      0.81      0.80      7865

request
accuracy_score  0.8921805467260012
classification_report 
               precision    recall  f1-score   support

           0       0.91      0.97      0.94      6545
           1       0.77      0.50      0.61      1320

    accuracy                           0.89      7865
   macro avg       0.84      0.74      0.77      7865
weighted avg       0.88      0.89      0.88      7865

offer
accuracy_score  0.9952956134774317
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00     

accuracy_score  0.9476160203432931
classification_report 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      7458
           1       0.44      0.05      0.08       407

    accuracy                           0.95      7865
   macro avg       0.70      0.52      0.53      7865
weighted avg       0.92      0.95      0.93      7865

direct_report
accuracy_score  0.8569612205975843
classification_report 
               precision    recall  f1-score   support

           0       0.87      0.97      0.92      6353
           1       0.74      0.39      0.51      1512

    accuracy                           0.86      7865
   macro avg       0.81      0.68      0.71      7865
weighted avg       0.85      0.86      0.84      7865



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# Train a model with KNN algorithms
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])
pipeline2.fit(X_train, y_train)
y_pred2=pipeline2.predict(X_test)
for i in range(36):
    print(columns[i])
    print('accuracy_score ',accuracy_score(y_test.T[i],y_pred2.T[i]))
    print('classification_report \n', classification_report(y_test.T[i],y_pred2.T[i]))

related
accuracy_score  0.778893833439288
classification_report 
               precision    recall  f1-score   support

           0       0.60      0.32      0.42      1870
           1       0.81      0.93      0.86      5934
           2       0.35      0.38      0.37        61

    accuracy                           0.78      7865
   macro avg       0.59      0.54      0.55      7865
weighted avg       0.76      0.78      0.75      7865

request
accuracy_score  0.842085187539733
classification_report 
               precision    recall  f1-score   support

           0       0.84      1.00      0.91      6545
           1       0.84      0.07      0.14      1320

    accuracy                           0.84      7865
   macro avg       0.84      0.54      0.52      7865
weighted avg       0.84      0.84      0.78      7865

offer
accuracy_score  0.9952956134774317
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00      1

accuracy_score  0.8151303242212333
classification_report 
               precision    recall  f1-score   support

           0       0.82      1.00      0.90      6353
           1       0.82      0.05      0.09      1512

    accuracy                           0.82      7865
   macro avg       0.82      0.52      0.50      7865
weighted avg       0.82      0.82      0.74      7865



Compare the results, I will choose random forest as the algorithms to use.

### 9. Export your model as a pickle file

In [15]:
# Pickle best model
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.